$\textbf{IT127-1L - SOFTWARE DEVELOPMENT LABORATORY} \\ \texttt{SY2425}$

NAME: MA. ADDINE ANNE T. CARREON
 
SECTION: AM14

In [1]:
from tkinter import Tk, ttk, Button, StringVar, Toplevel, Label, Entry, messagebox, Frame
import sqlite3
from datetime import datetime
import time

def create_table_if_not_exists():
    with sqlite3.connect('myclassmates.db') as conn:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS students (
                student_number TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                birthday TEXT,
                email TEXT,
                contact_number TEXT
            )
        """)
        conn.commit()

class AppPage:
    def __init__(self, root):
        self.root = root
        self.root.title("Classmates App")

        self.tree_style = ttk.Style()
        self.tree_style.theme_use('clam')
        self.tree_style.configure("Treeview",
                                  background="#E5D8F6", 
                                  foreground="black",
                                  rowheight=25,
                                  fieldbackground="#E5D8F6")  
        self.tree_style.map("Treeview",
                            background=[("selected", "#a66bc8")],  
                            foreground=[("selected", "white")])  
        
        # App View Setting up
        self.tree = ttk.Treeview(root, show='headings', 
                                 columns=("student_number", "name", "birthday", "email", "contact_number"))
        self.tree.heading("student_number", text="Student Number")
        self.tree.heading("name", text="Name")
        self.tree.heading("birthday", text="Birthday")
        self.tree.heading("email", text="Mapúa Email")
        self.tree.heading("contact_number", text="Contact Number")
        
        # Column Width
        self.tree.column("student_number", width=150)
        self.tree.column("name", width=200)
        self.tree.column("birthday", width=150)
        self.tree.column("email", width=250)
        self.tree.column("contact_number", width=150)

        self.tree.grid(row=0, column=0, rowspan=7, pady=6, padx=10)

        self.search_var = StringVar()
        search_frame = Frame(root)
        search_frame.grid(row=0, column=1, padx=5, sticky='e')

        search_entry = Entry(search_frame, textvariable=self.search_var)
        search_entry.pack(side="left")

        clear_button = Button(search_frame, text="X", command=self.clear_search, relief="flat")
        clear_button.pack(side="left", padx=(5, 0))

        search_button = Button(search_frame, text="🔍", command=self.search_student, relief="flat")
        search_button.pack(side="left")

        # Buttons
        button_width = 15
        add_button = Button(root, text="Add Student", width=button_width, command=self.open_add_window, 
                            bg="#a66bc8", fg="white") 
        add_button.grid(row=8, column=0, pady=5)

        edit_button = Button(root, text="Edit Student", width=button_width, command=self.open_edit_window, 
                             bg="#a66bc8", fg="white")
        edit_button.grid(row=9, column=0, pady=5)

        delete_button = Button(root, text="Delete Student", width=button_width, command=self.delete_student, 
                               bg="#a66bc8", fg="white") 
        delete_button.grid(row=10, column=0, pady=5)

        self.load_data()
        self.tree.bind("<Double-1>", self.on_double_click)

    def execute_query(self, query, params=(), retry_count=3):
        """Executes a query with retry mechanism for database locks."""
        for attempt in range(retry_count):
            try:
                with sqlite3.connect('myclassmates.db') as conn:
                    cursor = conn.cursor()
                    cursor.execute(query, params)
                    conn.commit()
                return cursor
            except sqlite3.OperationalError as e:
                if "database is locked" in str(e):
                    time.sleep(1) 
                else:
                    raise e
        messagebox.showerror("Database Error", "Database is locked. Please try again later.")

    def load_data(self):
        for i in self.tree.get_children():
            self.tree.delete(i)

        cursor = self.execute_query("SELECT * FROM students")
        rows = cursor.fetchall()
        for row in rows:
            self.tree.insert("", "end", values=row)

    def search_student(self):
        search_term = self.search_var.get().strip()
        if not search_term:
            self.load_data()
            return

        for i in self.tree.get_children():
            self.tree.delete(i)

        try:
            formatted_date = datetime.strptime(search_term, "%B %d, %Y").strftime("%Y-%m-%d")
        except ValueError:
            formatted_date = None

        cursor = self.execute_query("""
            SELECT * FROM students 
            WHERE LOWER(student_number) LIKE ? OR LOWER(name) LIKE ? OR 
                  LOWER(email) LIKE ? OR contact_number LIKE ? OR 
                  birthday LIKE ?
        """, ('%' + search_term.lower() + '%', '%' + search_term.lower() + '%', 
              '%' + search_term.lower() + '%', '%' + search_term + '%', 
              '%' + (formatted_date if formatted_date else search_term) + '%'))
        
        rows = cursor.fetchall()

        if rows:
            for row in rows:
                self.tree.insert("", "end", values=row)
        else:
            messagebox.showinfo("Search Result", "No Results Found")
        
    def clear_search(self):
        self.search_var.set("")
        self.load_data()
        
    def open_add_window(self):
        add_window = Toplevel(self.root)
        add_window.title("Add Student")

        Label(add_window, text="Student Number").grid(row=0, column=0)
        self.studnum_entry = Entry(add_window)
        self.studnum_entry.grid(row=0, column=1)

        Label(add_window, text="Name").grid(row=1, column=0)
        self.name_entry = Entry(add_window)
        self.name_entry.grid(row=1, column=1)

        Label(add_window, text="Birthday").grid(row=2, column=0)
        self.bday_entry = Entry(add_window)
        self.bday_entry.grid(row=2, column=1)

        Label(add_window, text="Mapúa Email").grid(row=3, column=0)
        self.email_entry = Entry(add_window)
        self.email_entry.grid(row=3, column=1)

        Label(add_window, text="Contact Number").grid(row=4, column=0)
        self.number_entry = Entry(add_window)
        self.number_entry.grid(row=4, column=1)
        
        Button(add_window, text="Add", command=self.add_student, 
               bg="#a66bc8", fg="white").grid(row=5, columnspan=2, pady=5)

    def add_student(self):
        student_number = self.studnum_entry.get()
        name = self.name_entry.get()
        birthday = self.bday_entry.get()
        email = self.email_entry.get()
        contact_number = self.number_entry.get()

        try:
            with sqlite3.connect('myclassmates.db') as conn:
                cursor = conn.cursor()
                cursor.execute("INSERT INTO students VALUES (?, ?, ?, ?, ?)",
                               (student_number, name, birthday, email, contact_number))
                conn.commit()

            self.tree.insert("", "end", values=(student_number, name, birthday, email, contact_number))
            self.clear_entries()

        except sqlite3.IntegrityError:
            messagebox.showwarning("Duplicate Entry", "Student number already exists.")
        except sqlite3.OperationalError:
            messagebox.showerror("Database Error", "The database is currently locked. Please try again later.")


    def clear_entries(self):
        """Clears the input fields for adding or editing a student."""
        self.studnum_entry.delete(0, 'end')
        self.name_entry.delete(0, 'end')
        self.bday_entry.delete(0, 'end')
        self.email_entry.delete(0, 'end')
        self.number_entry.delete(0, 'end')
        
    def delete_student(self):
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Delete Student", "Please select a student to delete.")
            return
        
        student_number = self.tree.item(selected_item)['values'][0] 
        self.execute_query("DELETE FROM students WHERE student_number = ?", (student_number,))
        self.tree.delete(selected_item)

    def open_edit_window(self):
        selected_item = self.tree.selection()
        if not selected_item:
            messagebox.showwarning("Edit Student", "Please select a student to edit.")
            return

        current_values = self.tree.item(selected_item)['values']
        student_number, name, birthday, email, contact_number = current_values

        edit_window = Toplevel(self.root)
        edit_window.title("Edit Student")

        Label(edit_window, text="Student Number").grid(row=0, column=0)
        self.edit_studnum_entry = Entry(edit_window)
        self.edit_studnum_entry.insert(0, student_number)
        self.edit_studnum_entry.grid(row=0, column=1)

        Label(edit_window, text="Name").grid(row=1, column=0)
        self.edit_name_entry = Entry(edit_window)
        self.edit_name_entry.insert(0, name)
        self.edit_name_entry.grid(row=1, column=1)

        Label(edit_window, text="Birthday").grid(row=2, column=0)
        self.edit_bday_entry = Entry(edit_window)
        self.edit_bday_entry.insert(0, birthday)
        self.edit_bday_entry.grid(row=2, column=1)

        Label(edit_window, text="Mapúa Email").grid(row=3, column=0)
        self.edit_email_entry = Entry(edit_window)
        self.edit_email_entry.insert(0, email)
        self.edit_email_entry.grid(row=3, column=1)

        Label(edit_window, text="Contact Number").grid(row=4, column=0)
        self.edit_number_entry = Entry(edit_window)
        self.edit_number_entry.insert(0, contact_number)
        self.edit_number_entry.grid(row=4, column=1)

        Button(edit_window, text="Save", command=lambda: self.save_edit(student_number),
               bg="#a66bc8", fg="white").grid(row=5, columnspan=2, pady=5)

    def save_edit(self, old_student_number):
        student_number = self.edit_studnum_entry.get()
        name = self.edit_name_entry.get()
        birthday = self.edit_bday_entry.get()
        email = self.edit_email_entry.get()
        contact_number = self.edit_number_entry.get()

        self.execute_query("""
            UPDATE students 
            SET student_number = ?, name = ?, birthday = ?, email = ?, contact_number = ? 
            WHERE student_number = ?
        """, (student_number, name, birthday, email, contact_number, old_student_number))

        self.load_data()
        messagebox.showinfo("Success", "Student information updated successfully.")

    def on_double_click(self, event):
        self.open_edit_window()

# Main Application Window
if __name__ == "__main__":
    root = Tk()
    app = AppPage(root)
    create_table_if_not_exists()
    root.mainloop()

***
***